# Prompt Chaining Example

This notebook demonstrates **Prompt Chaining**, a technique where complex tasks are broken into sequential prompts, with each step's output feeding into the next.

**The Concept:**
Instead of asking the AI to do everything in one go, you break the task into discrete steps. Each step has a focused objective, and the output becomes the input for the next step. This approach:
* Improves accuracy on complex, multi-part tasks
* Makes debugging easier (you can inspect each step)
* Allows different temperature/model settings per step
* Reduces context pollution and confusion

**Key Mechanics:**
1. **Decomposition**: Break the task into logical steps.
2. **Sequential Execution**: Run each step in order.
3. **Output Passing**: Use the output of step N as input to step N+1.
4. **Aggregation**: Combine results into a final output.

In [ ]:
%pip install openai python-dotenv --quiet

### 1. Setup and Authorization

We start by importing the necessary libraries and loading your OpenAI API key.

In [ ]:
from openai import OpenAI
import os
import time
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Paste your OpenAI API key: ").strip()

client = OpenAI(api_key=api_key)
print("OpenAI client ready!")

### 2. The Chain Execution Engine

This function runs a sequence of prompts, passing context between them.

In [ ]:
def run_chain(steps: list, initial_input: str, verbose: bool = True) -> dict:
    """
    Execute a chain of prompts sequentially.
    
    Args:
        steps: List of dicts with 'name', 'prompt_template', and optional 'temperature'
        initial_input: The starting input for the chain
        verbose: Whether to print intermediate results
    
    Returns:
        Dict with 'final_output' and 'step_outputs'
    """
    current_input = initial_input
    step_outputs = {}
    
    for i, step in enumerate(steps, 1):
        step_name = step.get('name', f'Step {i}')
        template = step['prompt_template']
        temperature = step.get('temperature', 0.7)
        
        if verbose:
            print(f"\n{'='*60}")
            print(f"STEP {i}: {step_name}")
            print(f"{'='*60}")
        
        # Format the prompt with available context
        prompt = template.format(
            input=current_input,
            initial_input=initial_input,
            **step_outputs
        )
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature
            )
            
            output = response.choices[0].message.content.strip()
            step_outputs[step_name.lower().replace(' ', '_')] = output
            current_input = output
            
            if verbose:
                print(f"\nOutput:\n{output[:500]}{'...' if len(output) > 500 else ''}")
            
            time.sleep(0.2)  # Rate limiting
            
        except Exception as e:
            print(f"Error at {step_name}: {e}")
            return {'final_output': None, 'step_outputs': step_outputs, 'error': str(e)}
    
    return {'final_output': current_input, 'step_outputs': step_outputs}

### 3. Example Chain: Blog Post Generator

This chain transforms a topic into a complete blog post through 4 steps:
1. **Research**: Identify key points and angles
2. **Outline**: Create a structured outline
3. **Draft**: Write the full content
4. **Polish**: Edit for clarity and engagement

In [ ]:
BLOG_POST_CHAIN = [
    {
        "name": "Research",
        "prompt_template": """Analyze this blog topic and identify:
1. The target audience and their pain points
2. 5 key points that must be covered
3. A unique angle or hook that makes this interesting
4. Potential examples or case studies to include

Topic: {input}

Provide a structured research brief.""",
        "temperature": 0.7
    },
    {
        "name": "Outline",
        "prompt_template": """Based on this research, create a detailed blog post outline:

Research:
{input}

Create an outline with:
- An attention-grabbing title
- Introduction hook
- 4-6 main sections with subpoints
- Conclusion with call-to-action

Format as a clear hierarchical outline.""",
        "temperature": 0.5
    },
    {
        "name": "Draft",
        "prompt_template": """Write a complete blog post based on this outline:

{input}

Guidelines:
- Write in a conversational, engaging tone
- Use short paragraphs for readability
- Include specific examples and actionable advice
- Target length: 800-1000 words

Write the complete blog post.""",
        "temperature": 0.8
    },
    {
        "name": "Polish",
        "prompt_template": """Edit and polish this blog post:

{input}

Improvements to make:
1. Strengthen the opening hook
2. Improve transitions between sections
3. Add a compelling call-to-action
4. Fix any awkward phrasing
5. Ensure consistent tone throughout

Return the polished, publication-ready blog post.""",
        "temperature": 0.4
    }
]

### 4. Example Chain: Code Review Pipeline

This chain performs a thorough code review through multiple lenses.

In [ ]:
CODE_REVIEW_CHAIN = [
    {
        "name": "Understand",
        "prompt_template": """Analyze this code and explain:
1. What the code does (high-level purpose)
2. The main components and their roles
3. The data flow through the code

Code:
{input}

Provide a clear technical summary.""",
        "temperature": 0.3
    },
    {
        "name": "Security Review",
        "prompt_template": """Perform a security review of this code:

Code Understanding:
{understand}

Original Code:
{initial_input}

Check for:
1. Input validation issues
2. Injection vulnerabilities (SQL, XSS, command)
3. Authentication/authorization flaws
4. Data exposure risks
5. Cryptographic weaknesses

List any security issues found with severity ratings.""",
        "temperature": 0.2
    },
    {
        "name": "Performance Review",
        "prompt_template": """Review this code for performance:

Code Understanding:
{understand}

Original Code:
{initial_input}

Analyze:
1. Time complexity of key operations
2. Memory usage patterns
3. Potential bottlenecks
4. Optimization opportunities

Provide specific recommendations.""",
        "temperature": 0.3
    },
    {
        "name": "Final Report",
        "prompt_template": """Create a comprehensive code review report:

## Code Summary
{understand}

## Security Findings
{security_review}

## Performance Findings
{performance_review}

Create a final report with:
1. Executive summary (2-3 sentences)
2. Critical issues to fix immediately
3. Recommended improvements (prioritized)
4. Overall code quality rating (1-10)

Format as a professional code review document.""",
        "temperature": 0.4
    }
]

### 5. Run the Chain

Choose a chain and provide your input to see prompt chaining in action.

In [ ]:
print("Prompt Chaining Examples")
print("1. Blog Post Generator")
print("2. Code Review Pipeline")

choice = input("\nSelect a chain (1-2): ").strip()

if choice == "1":
    chain = BLOG_POST_CHAIN
    print("\nBlog Post Generator selected.")
    user_input = input("Enter a blog topic: ").strip()
elif choice == "2":
    chain = CODE_REVIEW_CHAIN
    print("\nCode Review Pipeline selected.")
    print("Paste your code (enter a blank line when done):")
    lines = []
    while True:
        line = input()
        if not line:
            break
        lines.append(line)
    user_input = "\n".join(lines)
else:
    chain = BLOG_POST_CHAIN
    user_input = "How to stay productive while working from home"
    print(f"\nDefaulting to Blog Post Generator with topic: '{user_input}'")

if user_input:
    print("\nStarting chain execution...")
    result = run_chain(chain, user_input)
    
    print("\n" + "="*60)
    print("FINAL OUTPUT")
    print("="*60)
    print(result['final_output'])